In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Concatenate, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Loading the spotify dataset into pandas dataframe
# Rows with error are skipped and we choose the dataset in a random order
spotifyDataset = pd.read_csv('spotify_data_1m.csv', on_bad_lines='skip', skiprows=lambda i: i > 0 and random.random() > 0.95)

# Function to preprocess the textual data from the dataset
def preprocessTextData():
    # Genre data tokenized and padded according to max length
    textTokenizer = Tokenizer(num_words=5000)
    textTokenizer.fit_on_texts(spotifyDataset['genre'])
    genreSequences = textTokenizer.texts_to_sequences(spotifyDataset['genre'])
    maxSequenceLength = max(len(seq) for seq in genreSequences)
    paddedGenreSequences = pad_sequences(genreSequences, maxlen=maxSequenceLength, padding='post')
    
    return paddedGenreSequences, maxSequenceLength, textTokenizer

# Function to preprocess the numerical data from the dataset
def preprocessNumericData():
    # Numerical data scaled
    numericData = spotifyDataset.select_dtypes(include=np.number)
    standardScaler = StandardScaler()
    scaledNumericData = standardScaler.fit_transform(numericData)
    
    return scaledNumericData, standardScaler

paddedGenreSequences, maxSequenceLength, textTokenizer = preprocessTextData()
scaledNumericData, standardScaler = preprocessNumericData()

# Converting text and numeric sequences into np arrays
X_text = np.array(paddedGenreSequences)
X_numeric = np.array(scaledNumericData)

# Function to define the LSTM model with layers for both text and numerical data
def createModel():
    # Textual Layer
    textInputSequence = Input(shape=(maxSequenceLength,))
    embeddingLayer = Embedding(input_dim=5000, output_dim=16, input_length=maxSequenceLength)(textInputSequence)
    lstmLayer = LSTM(8, return_sequences=True, dropout=0.82, recurrent_dropout=0.82)(embeddingLayer)
    textOutputSequence = Flatten()(Dropout(0.82)(lstmLayer))

    # Numeric Layer
    numericInputSequence = Input(shape=(X_numeric.shape[1],))
    numericLayer = Dense(8, kernel_regularizer=l2(0.001))(numericInputSequence)
    numericOutputSequence = Dropout(0.82)(numericLayer)
    
    return textInputSequence, numericInputSequence, textOutputSequence, numericOutputSequence

textInputSequence, numericInputSequence, textOutputSequence, numericOutputSequence = createModel()

combinedOutputSequence = Concatenate()([textOutputSequence, numericOutputSequence])
combinedInputSequence = Dense(1, activation='tanh')(combinedInputSequence)

# Initiating the model
model = Model(inputs=[textInputSequence, numericInputSequence], outputs=combinedOutputSequence)

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy']) #use 'adamax' optimizer for 0.957

# Splitting the dataset into training and validation set
y = (spotifyDataset['popularity'] > spotifyDataset['popularity'].median()).astype(int)
    
X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(
    X_text, X_numeric, y, test_size=0.2, random_state=42
)

# Training the model
model.fit([X_train_text, X_train_numeric], y_train, epochs=10, batch_size=32, 
          validation_data=([X_test_text, X_test_numeric], y_test))

Epoch 1/10
27540/27540 [==============================] - 106s 4ms/step - loss: 0.1203 - accuracy: 0.8650 - val_loss: 0.0681 - val_accuracy: 0.9598
Epoch 2/10
27540/27540 [==============================] - 98s 4ms/step - loss: 0.1079 - accuracy: 0.8788 - val_loss: 0.0691 - val_accuracy: 0.9560
Epoch 3/10
27540/27540 [==============================] - 98s 4ms/step - loss: 0.1075 - accuracy: 0.8791 - val_loss: 0.0686 - val_accuracy: 0.9549
Epoch 4/10
27540/27540 [==============================] - 93s 3ms/step - loss: 0.1073 - accuracy: 0.8790 - val_loss: 0.0667 - val_accuracy: 0.9569
Epoch 5/10
27540/27540 [==============================] - 92s 3ms/step - loss: 0.1075 - accuracy: 0.8791 - val_loss: 0.0676 - val_accuracy: 0.9571
Epoch 6/10
27540/27540 [==============================] - 99s 4ms/step - loss: 0.1073 - accuracy: 0.8790 - val_loss: 0.0671 - val_accuracy: 0.9572
Epoch 7/10
27540/27540 [==============================] - 100s 4ms/step - loss: 0.1073 - accuracy: 0.8791 - val_loss:

In [3]:
# Evaluating the model
eval_result = model.evaluate([X_test_text, X_test_numeric], y_test)
accuracy = eval_result[1]

# Making predictions
y_pred = model.predict([X_test_text, X_test_numeric])
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculating metrics
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Accuracy: {:.6f}%".format(accuracy * 100))
print("Precision: {:.6f}%".format(precision * 100))
print("Recall: {:.6f}%".format(recall * 100))
print("F1-Score: {:.6f}%".format(f1 * 100))

6885/6885 [==============================] - 7s 1ms/step
Accuracy: 95.351738%
Precision: 92.944362%
Recall: 98.007829%
F1-Score: 95.408962%


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences

def recommendTopSongs(input_track_name, input_artist, input_energy, input_loudness, input_acousticness, input_tempo, input_duration_ms, model, textTokenizer, standardScaler):
    # Preprocessing input data for both text and numerical data
    genreInputSequence = textTokenizer.texts_to_sequences([input_artist])
    genreInputPaddedSequence = pad_sequences(genreInputSequence, maxlen=maxSequenceLength, padding='post')

    numericInputData = np.array([[input_energy, input_loudness, input_acousticness, input_tempo, input_duration_ms, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
    scaledNumericInputData = standardScaler.transform(numericInputData)

    # Predicting the popularity of the input song
    inputPrediction = model.predict([genreInputPaddedSequence, np.array(scaledNumericInputData)])

    # Extracting the genre and numeric data from all songs for recommendation
    artistGenreList = spotifyDataset['artist_name']
    trackNumericData = standardScaler.transform(spotifyDataset.select_dtypes(include=np.number))

    artistGenreSequence = textTokenizer.texts_to_sequences(artistGenreList)
    artistGenrePaddedSequence = pad_sequences(artistGenreSequence, maxlen=maxSequenceLength, padding='post')

    # Predicting popularity for all songs
    popularityPredictionList = model.predict([artistGenrePaddedSequence, np.array(trackNumericData)])

    # Calculating cosine similarity between the input song and all other songs
    cosineSimilarity = cosine_similarity(inputPrediction, popularityPredictionList).flatten()

    # Fetching top 3 songs
    recommendationCount = min(3, len(spotifyDataset) - 1)
    selectedIndices = np.argsort(cosineSimilarity)[-recommendationCount:][::-1]
    finalRecommendations = spotifyDataset[['track_name', 'artist_name', 'genre']].iloc[selectedIndices]

    return finalRecommendations

# Giving input song and its corresponding values (Using a random song from the dataset)
song = spotifyDataset.sample(n=1)
input_track_name = song['track_name'].values[0]
input_artist = song['artist_name'].values[0]
input_energy = song['energy'].values[0]
input_loudness = song['loudness'].values[0]
input_acousticness = song['acousticness'].values[0]
input_tempo = song['tempo'].values[0]
input_duration_ms = song['duration_ms'].values[0]

top3Songs = recommendTopSongs(input_track_name, input_artist, input_energy, input_loudness, input_acousticness, input_tempo, input_duration_ms, model, textTokenizer, standardScaler)

print(f"Input Track Name: {input_track_name}")
print(f"Input Artist Name: {input_artist}")
print("-------------------------------------------------")
print("Top 3 Recommended Tracks:\n")
i=0
for _, (song, artist, genre) in top3Songs.iterrows():
    i=i+1
    print(f"{i}) {song} by {artist} (Genre: {genre})")

1/1 [==============================] - 0s 18ms/step


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


34425/34425 [==============================] - 33s 964us/step
Input Track Name: Lost in Translation /1
Input Artist Name: infinite bisous
-------------------------------------------------
Top 3 Recommended Tracks:

1) Hit the Road Jack (Pé Na Éstrada) by Mo' Horizons (Genre: trip-hop)
2) Castigando by Max e Luan (Genre: forro)
3) Casal Raiz - Ao Vivo by Xand Avião (Genre: forro)
